In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import openai
from dotenv import load_dotenv, find_dotenv

In [2]:
# Numpy & Pandas tweaks for display
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [3]:
# load OPENAI_API_KEY from .env file
_ = load_dotenv(find_dotenv())

### Download Financial & historical stock data

In [4]:
def download_company_data(symbol: str, period: str = "5y"):
    ticker = yf.Ticker(symbol)
    prices = ticker.history(period=period)

    # fetch financial statements
    # income_stmt = ticker.income_stmt
    # balance_sheet = ticker.balance_sheet
    # cash_flow = ticker.cash_flow

    # download historical prices data (for last 5 years from today)
    # from datetime import datetime, timedelta

    # end_date = datetime.today()
    # # start_date is 5 years previous to
    # start_date = end_date - timedelta(days=5 * 365)
    # market_data = ticker.history(
    #     start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d")
    # )

    return ticker, prices

### Financial Parameter Analysis
A financial house will typically conduct a thorough Financial Parameter analysis of a target company to assess its financial health, stability and growth potential. These parameters are typically assesses from company's historical stock prices, its balance sheet, income statement, cash flow statement etc. 

In this section, we will calculate some sample parameters such as:
* Profitability Ratios:
    * Liquidity Ratios, such as Net Profit Margin and Return on Equity

<< TODO: complete this comment>>

#### Calculate Profitability Ratios
In this code fragment we'll calculate like **Net Profit Margin**, **Return on Equity (ROE)** and **Return on Assets (ROA)**

In [5]:
def calculate_profitability_ratios(symbol):
    # Fetch ticker data for symbol
    ticker, _ = download_company_data(symbol)

    # Fetch annual financials (income statement and balance sheet)
    financials = ticker.financials
    income_statement = ticker.financials
    balance_sheet = ticker.balance_sheet

    # Extract net income, total revenue and total assets for past 5 years
    net_income = income_statement.loc["Net Income"]
    revenue = financials.loc["Total Revenue"]
    total_assets = balance_sheet.loc["Total Assets"]
    shareholders_equity = balance_sheet.loc["Stockholders Equity"]

    # Calculate Net Profit Margin
    net_profit_margin = (net_income / revenue) * 100

    # Calculate ROE (Return on Equity)
    roe = (net_income / shareholders_equity) * 100

    # Calculate ROA for each year
    roa = net_income / total_assets

    # Prepare a DataFrame with the relevant data
    profitability_ratios = pd.DataFrame(
        {
            "Year": net_income.index.year,
            "Net Income": net_income.values,
            "Revenue": revenue.values,
            "Shareholders Equity": shareholders_equity.values,
            "Net Profit Margin (%)": net_profit_margin.values,
            "ROE (%)": roe.values,
            "Total Assets": total_assets.values,
            "ROA": roa.values,
        }
    )
    return profitability_ratios

In [6]:
profitability_ratios = calculate_profitability_ratios("MSFT")
profitability_ratios

,Year,Net Income,Revenue,Shareholders Equity,Net Profit Margin (%),ROE (%),Total Assets,ROA
0,2024,88136000000.0,245122000000.0,268477000000.0,35.955973,32.828138,512163000000.0,0.172086
1,2023,72361000000.0,211915000000.0,206223000000.0,34.146238,35.088715,411976000000.0,0.175644
2,2022,72738000000.0,198270000000.0,166542000000.0,36.686337,43.675469,364840000000.0,0.19937
3,2021,61271000000.0,168088000000.0,141988000000.0,36.45174,43.152238,333779000000.0,0.183568


#### Calculate Liquidity Ratios
In this code fragment we'll calculate like **Current Ratio** and **Quick Ratio**

In [7]:
def calculate_liquidity_ratios(symbol):
    # Fetch ticker data for symbol
    ticker, _ = download_company_data(symbol)
    balance_sheet = ticker.balance_sheet

    # Extract relevant data from the balance sheet
    current_assets = balance_sheet.loc["Current Assets"]
    current_liabilities = balance_sheet.loc["Current Liabilities"]
    # inventory = balance_sheet.loc["Inventory"]
    # Handle cases where inventory is zero or not listed
    inventory = (
        balance_sheet.loc["Inventory"]
        if "Inventory" in balance_sheet.index
        else pd.Series([0] * len(current_assets), index=current_assets.index)
    )

    # Calculate Current Ratio
    current_ratio = current_assets / current_liabilities

    # Calculate Quick Ratio
    quick_ratio = (current_assets - inventory) / current_liabilities

    # Prepare a DataFrame with the relevant data
    liquidity_metrics = pd.DataFrame(
        {
            "Year": current_assets.index.year,
            "Current Assets": current_assets.values,
            "Current Liabilities": current_liabilities.values,
            "Inventory": inventory.values,
            "Current Ratio": current_ratio.values,
            "Quick Ratio": quick_ratio.values,
        }
    )

    # Display the liquidity metrics
    return liquidity_metrics

In [8]:
liquidity_ratios = calculate_liquidity_ratios("MSFT")
liquidity_ratios

,Year,Current Assets,Current Liabilities,Inventory,Current Ratio,Quick Ratio
0,2024,159734000000.0,125286000000.0,1246000000.0,1.274955,1.26501
1,2023,184257000000.0,104149000000.0,2500000000.0,1.769167,1.745163
2,2022,169684000000.0,95082000000.0,3742000000.0,1.784607,1.745251
3,2021,184406000000.0,88657000000.0,2636000000.0,2.079994,2.050261


#### Calculate Leverage Ratios
In this code fragment we'll calculate like **Debt-to-Equity Ratio** and **Interest Coverage Ratio**

In [9]:
def calculate_leverage_ratios(symbol):
    # Fetch ticker data for symbol
    ticker, _ = download_company_data(symbol)

    balance_sheet = ticker.balance_sheet
    financials = ticker.financials

    # Extract relevant data from the balance sheet and income statement
    total_liabilities = balance_sheet.loc["Current Liabilities"]
    shareholders_equity = balance_sheet.loc["Stockholders Equity"]
    ebit = financials.loc["EBIT"]  # Earnings Before Interest and Taxes
    interest_expense = (
        financials.loc["Interest Expense"]
        if "Interest Expense" in financials.index
        else pd.Series([0] * len(ebit), index=ebit.index)
    )  # Han-dle cases where interest expense is zero or not listedExpense' in finan-cials.index else pd.Series([0] * len(ebit), index=ebit.index) # Handle cases where interest expense is zero or not listed

    # Calculate Debt-to-Equity Ratio
    debt_to_equity_ratio = total_liabilities / shareholders_equity

    # Calculate Interest Coverage Ratio
    interest_coverage_ratio = ebit / interest_expense.replace(
        0, float("nan")
    )  # Avoid division by zero by replacing 0 with NaN

    # Prepare a DataFrame with the relevant data
    leverage_ratios = pd.DataFrame(
        {
            "Year": total_liabilities.index.year,
            "Total Liabilities": total_liabilities.values,
            "Shareholders Equity": shareholders_equity.values,
            "EBIT": ebit.values,
            "Interest Expense": interest_expense.values,
            "Debt-to-Equity Ratio": debt_to_equity_ratio.values,
            "Interest Coverage Ratio": interest_coverage_ratio.values,
        }
    )

    # Display the financial ratios
    return leverage_ratios

In [10]:
leverage_ratios = calculate_leverage_ratios("MSFT")
leverage_ratios

,Year,Total Liabilities,Shareholders Equity,EBIT,Interest Expense,Debt-to-Equity Ratio,Interest Coverage Ratio
0,2024,125286000000.0,268477000000.0,110722000000.0,2935000000.0,0.466654,37.724702
1,2023,104149000000.0,206223000000.0,91279000000.0,1968000000.0,0.505031,46.381606
2,2022,95082000000.0,166542000000.0,85779000000.0,2063000000.0,0.570919,41.579738
3,2021,88657000000.0,141988000000.0,73448000000.0,2346000000.0,0.624398,31.307758


#### Efficiency Ratios
In this code fragment we'll calculate Efficiency Ratios such as **Asset Turnover Ratio** and **Inventory Turnover Ratio**

In [11]:
def calculate_efficiency_ratios(symbol):
    # Fetch ticker data for symbol
    ticker, _ = download_company_data(symbol)

    financials = ticker.financials
    balance_sheet = ticker.balance_sheet

    # Extract relevant data from the financial statements
    revenue = financials.loc["Total Revenue"]
    cogs = financials.loc["Cost Of Revenue"]
    total_assets = balance_sheet.loc["Total Assets"]
    inventory = (
        balance_sheet.loc["Inventory"]
        if "Inventory" in balance_sheet.index
        else pd.Series([0] * len(total_assets), index=total_assets.index)
    )  # Handle cases where inventory is zero or not listed

    # Calculate Average Total Assets for Asset Turnover Ratio
    average_total_assets = (total_assets + total_assets.shift(1)) / 2

    # Calculate Asset Turnover Ratio
    asset_turnover_ratio = revenue / average_total_assets

    # Calculate Average Inventory for Inventory Turnover Ratio
    average_inventory = (inventory + inventory.shift(1)) / 2

    # Calculate Inventory Turnover Ratio
    inventory_turnover_ratio = cogs / average_inventory.replace(
        0, float("nan")
    )  # Avoid division by zero by replacing 0 with NaN

    # Prepare a DataFrame with the relevant data
    efficiency_ratios = pd.DataFrame(
        {
            "Year": revenue.index.year,
            "Revenue": revenue.values,
            "COGS": cogs.values,
            "Total Assets": total_assets.values,
            "Inventory": inventory.values,
            "Asset Turnover Ratio": asset_turnover_ratio.values,
            "Inventory Turnover Ratio": inventory_turnover_ratio.values,
        }
    )

    # Display the turnover ratios
    return efficiency_ratios

In [12]:
efficiency_ratios = calculate_efficiency_ratios("MSFT")
efficiency_ratios

,Year,Revenue,COGS,Total Assets,Inventory,Asset Turnover Ratio,Inventory Turnover Ratio
0,2024,245122000000.0,74114000000.0,512163000000.0,1246000000.0,NaN,NaN
1,2023,211915000000.0,65863000000.0,411976000000.0,2500000000.0,0.458621,35.164442
2,2022,198270000000.0,62650000000.0,364840000000.0,3742000000.0,0.510468,20.073694
3,2021,168088000000.0,52232000000.0,333779000000.0,2636000000.0,0.481201,16.378802


#### Valuation Ratios
In this code fragment we'll calculate Valuation Ratios such as **P/E Ratio** and **EV/EBITDA Ratio**

In [13]:
def calculate_valuation_ratios(symbol):
    # Fetch ticker data for symbol
    ticker, prices = download_company_data("MSFT")

    # Fetch historical market price and calculate EPS
    prices = ticker.history(period="5y")
    financials = ticker.financials
    balance_sheet = ticker.balance_sheet

    # Extract relevant data
    net_income = financials.loc["Net Income"]
    shares_outstanding = ticker.info["sharesOutstanding"]
    eps = net_income / shares_outstanding

    # resample prices at month-end interval, which is same as that of eps
    market_price_per_share = prices["Close"].resample("ME").last()
    # convert index from tz-aware to tz-naive (i.e. does not have local time component)
    market_price_per_share.index = market_price_per_share.index.tz_convert(None)
    market_price_per_share.index = market_price_per_share.index.normalize()
    # grab the market_price values for dates in eps' index
    mps2 = market_price_per_share.loc[eps.index]

    # Calculate P/E Ratio
    pe_ratio = mps2 / eps

    ebitda = financials.loc["EBITDA"]  # Use EBITDA from financials
    total_debt = (
        balance_sheet.loc["Long Term Debt"] + balance_sheet.loc["Short Long Term Debt"]
        if "Short Long Term Debt" in balance_sheet.index
        else balance_sheet.loc["Long Term Debt"]
    )
    cash = (
        balance_sheet.loc["Cash"]
        if "Cash" in balance_sheet.index
        else pd.Series([0] * len(total_debt), index=total_debt.index)
    )  # Handle cases where cash is zero or not listed

    # market_cap = market_price_per_share * shares_outstanding
    market_cap = mps2 * shares_outstanding
    enterprise_value = market_cap + total_debt - cash
    ev_ebitda_ratio = enterprise_value / ebitda

    # Prepare a DataFrame with the relevant data
    valuation_ratios = pd.DataFrame(
        {
            "Year": eps.index.year,
            "Market Price per Share": mps2.values,
            "EPS": eps.values,
            "P/E Ratio": pe_ratio.values,
            "EBITDA": ebitda.values,
            "Total Debt": total_debt.values,
            "Cash": cash.values,
            "Enterprise Value": enterprise_value.values,
            "EV/EBITDA Ratio": ev_ebitda_ratio.values,
        }
    )

    # Display the valuation ratios
    return valuation_ratios

In [14]:
valuation_ratios = calculate_valuation_ratios("MSFT")
valuation_ratios

,Year,Market Price per Share,EPS,P/E Ratio,EBITDA,Total Debt,Cash,Enterprise Value,EV/EBITDA Ratio
0,2024,446.145874,11.857329,37.626169,133009000000.0,42688000000.0,0,3358908070344.5,25.253239
1,2023,337.288818,9.735048,34.646857,105140000000.0,41990000000.0,0,2549071235245.0,24.244543
2,2022,251.942673,9.785767,25.745827,100239000000.0,47032000000.0,0,1919731931856.5625,19.151547
3,2021,263.592499,8.243061,31.977502,85134000000.0,50074000000.0,0,2009367553386.625,23.60241


#### Cash Flow Analysis
In this section, we will calculate metrics such as **Operating Cash Flow** and **Free Cash Flow**

In [15]:
def calculate_cashflow_metrics(symbol):
    # Fetch ticker data for symbol
    ticker, _ = download_company_data("MSFT")

    # NOTE: cash flow is showing 1 more column than rest of the metrics, hence dropping 2020-06-30
    cash_flow_cols = ticker.cash_flow.columns.drop(["2020-06-30"])
    cashflow = ticker.cash_flow[cash_flow_cols]

    # Extract relevant data from the cash flow statement
    net_income = ticker.income_stmt.loc["Net Income"]
    depreciation = cashflow.loc["Depreciation"]
    changes_in_working_capital = cashflow.loc["Change In Working Capital"]
    capex = cashflow.loc["Capital Expenditure"]

    # Calculate Operating Cash Flow
    operating_cash_flow = net_income + depreciation + changes_in_working_capital

    # Calculate Free Cash Flow
    free_cash_flow = operating_cash_flow - capex

    # Prepare a DataFrame with the relevant data
    cash_flow_metrics = pd.DataFrame(
        {
            "Year": operating_cash_flow.index.year,
            "Net Income": net_income.values,
            "Depreciation": depreciation.values,
            "Changes in Working Capital": changes_in_working_capital.values,
            "Operating Cash Flow (OCF)": operating_cash_flow.values,
            "Capital Expenditures (CapEx)": capex.values,
            "Free Cash Flow (FCF)": free_cash_flow.values,
        }
    )

    # Display the cash flow metrics
    return cash_flow_metrics

In [16]:
cash_flow_metrics = calculate_cashflow_metrics("MSFT")
cash_flow_metrics

,Year,Net Income,Depreciation,Changes in Working Capital,Operating Cash Flow (OCF),Capital Expenditures (CapEx),Free Cash Flow (FCF)
0,2024,88136000000.0,22287000000.0,1824000000.0,112247000000.0,-44477000000.0,156724000000.0
1,2023,72361000000.0,13861000000.0,-2388000000.0,83834000000.0,-28107000000.0,111941000000.0
2,2022,72738000000.0,14460000000.0,446000000.0,87644000000.0,-23886000000.0,111530000000.0
3,2021,61271000000.0,11686000000.0,-936000000.0,72021000000.0,-20622000000.0,92643000000.0


#### Dividend Analysis
In this section, we'll calculate ratios such as **Dividend Yield** and **Dividend Payout Ratio**

In [17]:
import yfinance as yf
import pandas as pd

# Fetch Microsoft stock data for the past 5 years
ticker = yf.Ticker("MSFT")

# Fetch historical stock prices and dividends data
prices = ticker.history(period="5y")
dividends = ticker.dividends

# Fetch annual financials: Net Income and calculate Total Dividends Paid
annual_financials = ticker.financials
net_income = ticker.income_stmt.loc["Net Income"]
shares_outstanding = ticker.info["sharesOutstanding"]

# Calculate Annual Dividends per Share by resampling dividends to yearly and sum-ming
annual_dividends_per_share = dividends.resample("YE").sum()

# Calculate Market Price per Share (use the last closing price of each year)
# market_price_per_share = prices["Close"].resample("YE").last()

# resample prices at month-end interval, which is same as that of eps
market_price_per_share = prices["Close"].resample("ME").last()
# convert index from tz-aware to tz-naive (i.e. does not have local time component)
market_price_per_share.index = market_price_per_share.index.tz_convert(None)
market_price_per_share.index = market_price_per_share.index.normalize()
# grab the market_price values for dates in eps' index
mps2 = market_price_per_share.loc[net_income.index]

# Calculate Total Dividends Paid (Annual Dividends per Share * Shares Outstanding)
total_dividends_paid = annual_dividends_per_share * shares_outstanding

# Calculate Dividend Yield
dividend_yield = (annual_dividends_per_share / market_price_per_share) * 100

# Calculate Dividend Payout Ratio
dividend_payout_ratio = (total_dividends_paid / net_income) * 100

# Prepare a DataFrame with the relevant data
dividend_metrics = pd.DataFrame(
    {
        "Year": annual_dividends_per_share.index.year,
        "Net Income": net_income.values,
        "Total Dividends Paid": total_dividends_paid.values,
        "Annual Dividends per Share": annual_dividends_per_share.values,
        "Market Price per Share": market_price_per_share.values,
        "Dividend Yield (%)": dividend_yield.values,
        "Dividend Payout Ratio (%)": dividend_payout_ratio.values,
    }
)

# Display the dividend metrics
dividend_metrics

TypeError: Cannot join tz-naive with tz-aware DatetimeIndex

In [22]:
# Fetch Microsoft stock data for the past 5 years
ticker = yf.Ticker("MSFT")

# Fetch historical stock prices and dividends data
prices = ticker.history(period="5y")
dividends = ticker.dividends

# Fetch annual financials: Net Income and calculate Total Dividends Paid
annual_financials = ticker.financials
net_income = ticker.income_stmt.loc["Net Income"]
shares_outstanding = ticker.info["sharesOutstanding"]

# Calculate Annual Dividends per Share by resampling dividends to yearly and sum-ming
annual_dividends_per_share = dividends.resample("ME").sum()

# Calculate Market Price per Share (use the last closing price of each year)
# market_price_per_share = prices["Close"].resample("YE").last()

# resample prices at month-end interval, which is same as that of eps
market_price_per_share = prices["Close"].resample("ME").last()
# convert index from tz-aware to tz-naive (i.e. does not have local time component)
market_price_per_share.index = market_price_per_share.index.tz_convert(None)
market_price_per_share.index = market_price_per_share.index.normalize()
# grab the market_price values for dates in eps' index
mps2 = market_price_per_share.loc[net_income.index]

# Calculate Total Dividends Paid (Annual Dividends per Share * Shares Outstanding)
total_dividends_paid = annual_dividends_per_share * shares_outstanding
# convert index from tz-aware to tz-naive (i.e. does not have local time component)
total_dividends_paid.index = total_dividends_paid.index.tz_convert(None)
total_dividends_paid.index = total_dividends_paid.index.normalize()
# grab the market_price values for dates in eps' index
tdp2 = total_dividends_paid.loc[net_income.index]
# total_dividends_paid = total_dividends_paid.loc[net_income.index]
mps2, tdp2

In [ ]:
# Calculate EV/EBITDA Ratio
ebitda = financials.loc["EBITDA"]  # Use EBITDA from financials
total_debt = (
    balance_sheet.loc["Long Term Debt"] + balance_sheet.loc["Short Long Term Debt"]
    if "Short Long Term Debt" in balance_sheet.index
    else balance_sheet.loc["Long Term Debt"]
)
cash = (
    balance_sheet.loc["Cash"]
    if "Cash" in balance_sheet.index
    else pd.Series([0] * len(total_debt), index=total_debt.index)
)  # Handle cases where cash is zero or not listed

# market_cap = market_price_per_share * shares_outstanding
market_cap = mps2 * shares_outstanding
enterprise_value = market_cap + total_debt - cash
ev_ebitda_ratio = enterprise_value / ebitda

df = pd.DataFrame(
    {
        "EBITDA": ebitda.values,
        "Total Debt": total_debt.values,
        "Cash": cash.values,
        "EV/EBIDTA": ev_ebitda_ratio.values,
    }
)
df.head(20)

In [16]:
cash

2024-06-30    0
2023-06-30    0
2022-06-30    0
2021-06-30    0
dtype: int64

In [67]:
market_price_per_share = prices["Close"].resample("ME").last()
market_price_per_share.index = market_price_per_share.index.tz_convert(None)
market_price_per_share.index = market_price_per_share.index.normalize()
# market_price_per_share.info()
market_price_per_share.loc[eps.index]

2024-06-30    446.145874
2023-06-30    337.288818
2022-06-30    251.942642
2021-06-30    263.592560
Name: Close, dtype: float64

In [57]:
# Define a dictionary containing employee data
data = {
    "Name": ["Jai", "Princi", "Gaurav", "Anuj", "Geeku"],
    "Age": [27, 24, 22, 32, 15],
    "Address": ["Delhi", "Kanpur", "Allahabad", "Kannauj", "Noida"],
    "Qualification": ["Msc", "MA", "MCA", "Phd", "10th"],
}

index = ["a", "b", "c", "d", "e"]

# Convert the dictionary into DataFrame
df = pd.DataFrame(data, index)
df

,Name,Age,Address,Qualification
a,Jai,27,Delhi,Msc
b,Princi,24,Kanpur,MA
c,Gaurav,22,Allahabad,MCA
d,Anuj,32,Kannauj,Phd
e,Geeku,15,Noida,10th


In [58]:
df.reset_index(inplace=True)
df

,index,Name,Age,Address,Qualification
0,a,Jai,27,Delhi,Msc
1,b,Princi,24,Kanpur,MA
2,c,Gaurav,22,Allahabad,MCA
3,d,Anuj,32,Kannauj,Phd
4,e,Geeku,15,Noida,10th


In [58]:
annual_dividends_per_share = ticker.dividends.resample("YE").sum()
close_prices_yearly = market_data["Close"].resample("YE").last()
dividend_yield = annual_dividends_per_share / close_prices_yearly
dividend_yield

Date
2019-12-31 00:00:00-05:00    0.003374
2020-12-31 00:00:00-05:00    0.009702
2021-12-31 00:00:00-05:00    0.007002
2022-12-31 00:00:00-05:00    0.010744
2023-12-31 00:00:00-05:00    0.007460
2024-12-31 00:00:00-05:00    0.005216
Freq: YE-DEC, dtype: float64

In [25]:
cash_flow.to_csv("~/Downloads/cash_flow_msft.csv", header=True, index=True)

In [5]:
def calculate_financial_ratios(symbol):
    ratios = {}

    # ticker, market_data, income_stmt, balance_sheet, cash_flow = download_company_data(
    #     symbol
    # )
    ticker, market_data = download_company_data(symbol)

    # calculate ratios

    # ---------- Profitability Ratios --------------
    # Net profit margin = Net Income / Total Revenue
    ratios["Net Profit Margin"] = (
        ticker.income_stmt.loc["Net Income"] / ticker.income_stmt.loc["Total Revenue"]
    )
    # Return on Equity (ROE) = Net Income / Total Stockhoder Equity
    ratios["ROE"] = (
        ticker.income_stmt.loc["Net Income"]
        / ticker.balance_sheet.loc["Stockholders Equity"]
    )
    # Return on Assets (ROA) = Net Income / Total Assets
    ratios["ROA"] = (
        ticker.income_stmt.loc["Net Income"] / ticker.balance_sheet.loc["Total Assets"]
    )

    # ---------- Liquidity Ratios --------------
    # Current Ratio = Current Assets / Current Liabilities
    ratios["Current Ratio"] = (
        ticker.balance_sheet.loc["Current Assets"]
        / ticker.balance_sheet.loc["Current Liabilities"]
    )
    # Quick Ratio = (Current Assets - Inventory) / Current Liabilities
    ratios["Quick Ratio"] = (
        ticker.balance_sheet.loc["Current Assets"]
        - ticker.balance_sheet.loc["Inventory"]
    ) / ticker.balance_sheet.loc["Current Liabilities"]

    # ---------- Leverage Ratios --------------
    # Debt to Equity = Total Liabilities / Stockholders Equity
    ratios["Debt To Equity"] = (
        ticker.balance_sheet.loc["Current Liabilities"]
        / ticker.balance_sheet.loc["Stockholders Equity"]
    )
    # Interest Coverage = EBIT / Interest Expense
    ratios["Interest Coverage"] = (
        ticker.income_stmt.loc["EBIT"] / ticker.income_stmt.loc["Interest Expense"]
    )

    # ---------- Efficiency Ratios --------------
    # Asset Turnover = Total Revenue / Total Assets
    ratios["Asset Turnover"] = (
        ticker.income_stmt.loc["Total Revenue"]
        / ticker.balance_sheet.loc["Total Assets"]
    )
    # Inventory Turnover = Cost of Revenue / Inventory
    ratios["Inventory Turnover"] = (
        ticker.income_stmt.loc["Cost Of Revenue"]
        / ticker.balance_sheet.loc["Inventory"]
    )

    # ---------- Valuation Metrics --------------
    # P/E Ratio = Market Price per Share / Earnings per Share (EPS)
    eps = ticker.financials.loc["Net Income"] / ticker.info["sharesOutstanding"]
    ratios["EPS"] = eps
    ratios["P/E Ratio"] = market_data["Close"].iloc[-1] / eps

    # ---------- Cash Flow Analysis --------------
    ratios["Operating Cash Flow"] = ticker.cash_flow.loc["Operating Cash Flow"]
    ratios["Free Cash Flow"] = (
        ticker.cash_flow.loc["Operating Cash Flow"]
        - ticker.cash_flow.loc["Capital Expenditure"]
    )

    # ---------- Divident Analysis --------------
    ratios["Dividend Yield"] = market_data

    return ratios


ratios = calculate_financial_ratios("MSFT")
ratios

{'Net Profit Margin': 2024-06-30     0.35956
 2023-06-30    0.341462
 2022-06-30    0.366863
 2021-06-30    0.364517
 dtype: object,
 'ROE': 2024-06-30    0.328281
 2023-06-30    0.350887
 2022-06-30    0.436755
 2021-06-30    0.431522
 dtype: object,
 'ROA': 2024-06-30    0.172086
 2023-06-30    0.175644
 2022-06-30     0.19937
 2021-06-30    0.183568
 dtype: object,
 'Current Ratio': 2024-06-30    1.274955
 2023-06-30    1.769167
 2022-06-30    1.784607
 2021-06-30    2.079994
 dtype: object,
 'Quick Ratio': 2024-06-30     1.26501
 2023-06-30    1.745163
 2022-06-30    1.745251
 2021-06-30    2.050261
 dtype: object,
 'Debt To Equity': 2024-06-30    0.466654
 2023-06-30    0.505031
 2022-06-30    0.570919
 2021-06-30    0.624398
 dtype: object,
 'Interest Coverage': 2024-06-30    37.724702
 2023-06-30    46.381606
 2022-06-30    41.579738
 2021-06-30    31.307758
 dtype: object,
 'Asset Turnover': 2024-06-30    0.478602
 2023-06-30    0.514387
 2022-06-30    0.543444
 2021-06-30    0

In [7]:
# Fetching hostorical market data (stocks info) for past 5 years
from datetime import datetime, timedelta

end_date = datetime.today()
# start_date is 5 years previous to
start_date = end_date - timedelta(days=5 * 365)
market_data = msft.history(
    start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d")
)
market_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-09-18 00:00:00-04:00,131.199396,132.450640,130.406619,132.307373,23982100,0.0,0.0
2019-09-19 00:00:00-04:00,134.007527,135.984679,133.787846,134.742996,35772100,0.0,0.0
2019-09-20 00:00:00-04:00,134.685674,135.296969,132.049466,133.186096,39167300,0.0,0.0
2019-09-23 00:00:00-04:00,132.985542,133.367611,132.230980,132.899582,17139300,0.0,0.0
2019-09-24 00:00:00-04:00,134.064817,134.380019,130.740900,131.218475,29773200,0.0,0.0
